In [1]:
from variables import *
import numpy as np

class Microgrid(object):
    def __init__(self,
        workingstatus=[0 ,0 ,0],
        #the working status of [ solar PV , wind turbine , generator]#
        SOC=0,
        #the state of charge of the battery system #
        actions_adjustingstatus=[0 ,0 ,0],
        #the actions of adjusting the working status(connected = 1 or not = 0 to the load)of the [solar , wind , generator ]#
        actions_solar=[0 ,0 ,0],
        #the solar energy used for supporting [the energy load, charging battery, sold back ]#
        actions_wind=[0 ,0 ,0],
        #the wind energy used for supporting [the energy load , charging battery , sold back ]#
        actions_generator=[0 ,0 ,0],
    dd    #the use of the energy generated by the generator for supporting [the energy load , charging battery , sold back ]#
        actions_purchased=[0 ,0],
        #the use of the energy purchased from the grid for supporting [the energy load , charging battery ]#
        actions_discharged=0,
        #the energy discharged by the battery for supporting the energy load #
        solarirradiance=0,
        #the environment feature : solar irradiance at current decision epoch #
        windspeed=0
        #the environment feature : wind speed at current decision epoch #
  ):
        self.workingstatus = workingstatus
        self.SOC = SOC
        self.actions_adjustingstatus = actions_adjustingstatus
        self.actions_solar = actions_solar
        self.actions_wind = actions_wind
        self.actions_generator = actions_generator
        self.actions_purchased = actions_purchased
        self.actions_discharged = actions_discharged
        self.solarirradiance = solarirradiance
        self.windspeed = windspeed
        self.Delta_t = 1

    def transition(self):
        workingstatus = self.workingstatus
        SOC = self.SOC
        if self.actions_adjustingstatus[1-1] == 1:
            workingstatus[1-1] = 1
        else:
            workingstatus[1-1] = 0
        # determining the next decision epoch working status of solar PV , 1= working , 0= not working #
        if self.actions_adjustingstatus[2-1] == 0 or self.windspeed > cutoff_windspeed or self.windspeed < cutin_windspeed:
            workingstatus[2-1] = 0
        else:
            if self.actions_adjustingstatus[2-1] == 1 and self.windspeed <= cutoff_windspeed and self.windspeed >= cutin_windspeed:
                workingstatus[2-1] = 1
        # determining the next decision epoch working status of wind turbine , 1= working , 0= not working #
        if self.actions_adjustingstatus[3-1] == 1:
            workingstatus[3-1] = 1
        else:
            workingstatus[3-1] = 0
        # determining the next decision epoch working status of generator , 1= working , 0= not working #
        SOC = self.SOC +(self.actions_solar[2 -1] + self.actions_wind[2-1] + self.actions_generator[2-1] + self.actions_purchased[2-1])* charging_discharging_efficiency - self.actions_discharged / charging_discharging_efficiency
        if SOC > SOC_max:
            SOC = SOC_max
        if SOC < SOC_min:
            SOC = SOC_min
        # determining the next desicion epoch SOC , state of charge of the battery system #
        return workingstatus , SOC

    def EnergyConsumption(self):
        # returns the energy consumption from the grid #
        return -(self.actions_solar[1-1] + self.actions_wind[1-1] + self.actions_generator[1-1] + self.actions_discharged)

    def energy_generated_solar(self):
        # calculate the energy generated by the solar PV , e_t ^ s #
        if self.workingstatus [1-1] == 1:
            energy_generated_solar = self.solarirradiance * area_solarPV * efficiency_solarPV / 1000
        else:
            energy_generated_solar = 0
        return energy_generated_solar

    def energy_generated_wind(self):
        # calculate the energy generated by the wind turbine , e_t ^ w #
        if self.workingstatus [2-1] == 1 and self.windspeed < rated_windspeed and self.windspeed >= cutin_windspeed:
            energy_generated_wind = number_windturbine * rated_power_wind_turbine *(self.windspeed - cutin_windspeed)/(rated_windspeed - cutin_windspeed)
        else:
            if self.workingstatus[2-1] == 1 and self.windspeed < cutoff_windspeed and self.windspeed >= rated_windspeed:
                energy_generated_wind = number_windturbine * rated_power_wind_turbine * self.Delta_t
            else:
                energy_generated_wind = 0
        return energy_generated_wind

    def energy_generated_generator(self):
        # calculate the energy generated bv the generator , e_t ^ g #
        if self.workingstatus[3-1] == 1:
            energy_generated_generator = number_generators * rated_output_power_generator * self.Delta_t
        else:
            energy_generated_generator = 0
        return energy_generated_generator

    def OperationalCost(self):
        # returns the operational cost for the onsite generation system #
        if self.workingstatus[1-1]==1:
            energy_generated_solar = self.solarirradiance * area_solarPV * efficiency_solarPV /1000
        else:
            energy_generated_solar = 0
        # calculate the energy generated by the solar PV , e_t ^ s #
        if self.workingstatus[2-1] == 1 and self.windspeed < rated_windspeed and self.windspeed >= cutin_windspeed:
            energy_generated_wind = number_windturbine * rated_power_wind_turbine * (self.windspeed - cutin_windspeed) / (rated_windspeed - cutin_windspeed)
        else:
            if self.workingstatus[2-1] == 1 and self.windspeed < cutoff_windspeed and self.windspeed >= rated_windspeed:
                energy_generated_wind = number_windturbine * rated_power_wind_turbine * self.Delta_t
            else:
                energy_generated_wind = 0
        # calculate the energy generated by the wind turbine , e_t ^ w #
        if self.workingstatus[3-1] == 1:
            energy_generated_generator = number_generators * rated_output_power_generator * self.Delta_t
        else:
            energy_generated_generator = 0
        # calculate the energy generated bv the generator , e_t ^ g #
        operational_cost = energy_generated_solar * unit_operational_cost_solar + energy_generated_wind * unit_operational_cost_wind + energy_generated_generator * unit_operational_cost_generator
        operational_cost+=(self.actions_discharged + self.actions_solar[2-1]+ self.actions_wind[2-1]+ self.actions_generator[2-1]) * self.Delta_t * unit_operational_cost_battery / (2* capacity_battery_storage * (SOC_max - SOC_min))
        # calculate the operational cost for the onsite generation system #
        return operational_cost

    def SoldBackReward(self):
        # calculate the sold back reward (benefit) #
        return(self.actions_solar[3-1] + self.actions_wind[3-1] + self.actions_generator[3-1]) * unit_reward_soldbackenergy

    def PrintMicrogrid(self , file):
        # print the current and the next states of the microgrid #
        print("Microgrid working status [solar PV, wind turbine, generator]= ", self.workingstatus, ", SOC = ", self.SOC, file=file)
        print("microgrid actions [solar PV, wind turbine, generator]= ", self.actions_adjustingstatus, file=file)
        print("solar energy supporting [the energy load, charging battery, sold back]= ", self.actions_solar, file=file)
        print("wind energy supporting [the energy load, charging battery, sold back]= ", self.actions_wind, file=file)
        print("generator energy supporting [the energy load, charging battery, sold back] = ", self.actions_generator, file=file)
        print("energy purchased from grid supporting [the energy load, charging battery] = ", self.actions_purchased, file=file)
        print("energy discharged by the battery supporting the energy load = ", self.actions_discharged, file=file)
        print("solar irradiance = ", self.solarirradiance, file=file)
        print("wind speed = ", self.windspeed, file=file)
        print("Microgrid Energy Consumption = ", self.EnergyConsumption(), file=file)
        print("Microgrid Operational Cost = ", self.OperationalCost(), file=file)
        print("Microgrid SoldBackReward = ", self.SoldBackReward(), file=file)
        print("\n", file=file)
        return None

if __name__ == '__main__':
    grid = Microgrid()
    grid.PrintMicrogrid(file=open("test.txt", 'w'))

NameError: name 'unit_reward_soldbackenergy' is not defined